<a href="https://colab.research.google.com/github/SimonHui2023/nlp-notebooks/blob/main/LLM_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Runtime type with A100/V100/T4 GPU
* go to “Change runtime type” under “Runtime” in the menu bar
* select “GPU” under “Hardware accelerator”
* Then “Save”




In [ ]:
!nvidia-smi

Sat Jan  6 14:07:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch, transformers
from transformers import AutoTokenizer
# check torch and transformers version
print(f"transformers version: {transformers.__version__}")
print(f"torch version: {torch.__version__}")
# check current device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

transformers version: 4.35.2
torch version: 2.1.0+cu121
cuda:0


## Reference links
* [大模型笔记！以LLAMA为例，快速入门LLM的推理过程 ](https://mp.weixin.qq.com/s/xPjuBitTw0c_kYy2zg2plw)
* [Transformer "Attention is all you need (1) ](https://mp.weixin.qq.com/s?__biz=MzkwMTU3NjYwOA==&mid=2247484109&idx=1&sn=f527a1786e96bf27c3fbd5a251e8c45d&chksm=c0b3e002f7c46914ad712e04e27ba692f05e3cdd1a2a64283eca56b873cf53d5f560fef7c891&scene=178&cur_album_id=3219479430338445314#rd)
* [Transformer "Attention is all you need (2) ](https://mp.weixin.qq.com/s?__biz=MzkwMTU3NjYwOA==&mid=2247484188&idx=1&sn=5a0428286160264c417757fd73775109&chksm=c0b3e1d3f7c468c505de525b53e7e308d56cbc8cff90f80564f86f5e01ec7a5991a33a953de5&scene=178&cur_album_id=3219479430338445314#rd)
* [Transformer "Attention is all you need (3) ](https://mp.weixin.qq.com/s?__biz=MzkwMTU3NjYwOA==&mid=2247484265&idx=1&sn=696877ff842a4abac64ff4053197817c&chksm=c0b3e1a6f7c468b0407d9066dd733fda81776c6d3fc96cd57c5d92de62995253dd9a4092472d&scene=178&cur_album_id=3219479430338445314#rd)
* [Transformer "Attention is all you need (4) ](https://mp.weixin.qq.com/s?__biz=MzkwMTU3NjYwOA==&mid=2247484306&idx=1&sn=c0e98c4dc10cc807097db44ee219a4f1&chksm=c0b3e15df7c4684b32437e53d4de360ebe130a918c894025595406b4978848b6bcb551a88234&scene=178&cur_album_id=3219479430338445314#rd)

## Microsoft phi-2 Tokenizer
[Phi-2](https://huggingface.co/microsoft/phi-2/tree/main) is a Transformer with 2.7 billion parameters. It was trained using the same data sources as Phi-1.5, augmented with a new data source that consists of various NLP synthetic texts and filtered websites (for safety and educational value). When assessed against benchmarks testing common sense, language understanding, and logical reasoning, Phi-2 showcased a nearly state-of-the-art performance among models with less than 13 billion parameters.

In [ ]:
# https://huggingface.co/microsoft/phi-2
model = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(
  model,
  use_fast=False,
  trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(tokenizer.vocab_size)

50257


In [ ]:
message="Hello World"
out = tokenizer.encode_plus(
  text=message,
  text_pair="",
  return_attention_mask=True,
  return_tensors="pt",
  add_special_tokens=True,
  return_length=True,
  return_special_tokens_mask=True,
  return_token_type_ids=True,
).to(device)
for k, v in out.items():
    print(k, ':', v)

input_ids : tensor([[15496,  2159]], device='cuda:0')
token_type_ids : tensor([[0, 0]], device='cuda:0')
special_tokens_mask : tensor([[0, 0]], device='cuda:0')
attention_mask : tensor([[1, 1]], device='cuda:0')
length : tensor([2], device='cuda:0')


In [ ]:
print(type(out['input_ids'][0]))
print(out['input_ids'].shape)
print(len(out['input_ids']))
print(tokenizer.convert_ids_to_tokens(out['input_ids'][0]))
print(tokenizer.decode(out['input_ids'][0]))

<class 'torch.Tensor'>
torch.Size([1, 2])
1
['Hello', 'ĠWorld']
Hello World


## Mistral Tokenizer
[The Mixtral-8x7B](https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1/tree/main) Large Language Model (LLM) is a pretrained generative Sparse Mixture of Experts. The Mixtral-8x7B outperforms Llama 2 70B on most benchmarks we tested.

For full details of this model please read our [release blog post](https://mistral.ai/news/mixtral-of-experts/).

In [ ]:
from transformers import AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)
messages = [
    {"role": "user", "content": "Tell me Mixtral."},
]

print("\n\n*** Prompt:")
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    return_tensors="pt",
)
print(tokenizer.decode(input_ids[0]))

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]



*** Prompt:
<s> [INST] Tell me Mixtral. [/INST]


## GPT Tokenizer
[tiktoken](https://github.com/openai/tiktoken)
tiktoken is a fast BPE tokeniser for use with OpenAI's models.

Reference: [How to count tokens with tiktoken](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb)

In [ ]:
!pip install tiktoken

In [ ]:
import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
# encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
message = "LLM is great!"
encoding.encode(message)

[4178, 44, 374, 2294, 0]

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(message, "cl100k_base")

5

In [ ]:
print(encoding.decode([4178, 44, 374, 2294, 0]))
print(encoding.decode([4178, 44, 374, 2294]))
print(encoding.decode([4178, 44, 374]))
print(encoding.decode([4178, 44]))
print(encoding.decode([4178]))
[encoding.decode_single_token_bytes(token) for token in [4178, 44, 374, 2294, 0]]

LLM is great!
LLM is great
LLM is
LLM
LL


[b'LL', b'M', b' is', b' great', b'!']

In [ ]:
def compare_encodings(example_string: str) -> None:
    """Prints a comparison of three string encodings."""
    # print the example string
    print(f'\nExample string: "{example_string}"')
    # for each encoding, print the # of tokens, the token integers, and the token bytes
    for encoding_name in ["r50k_base", "p50k_base", "cl100k_base"]:
        encoding = tiktoken.get_encoding(encoding_name)
        token_integers = encoding.encode(example_string)
        num_tokens = len(token_integers)
        token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]
        print()
        print(f"{encoding_name}: {num_tokens} tokens")
        print(f"token integers: {token_integers}")
        print(f"token bytes: {token_bytes}")

compare_encodings(message)
compare_encodings("Merry Christmas")
compare_encodings("春眠不觉晓")

[encoding.decode_single_token_bytes(token).decode('utf-8', errors='replace') for token in [11881, 98, 27280, 254, 16937, 19658, 231, 45114, 241]]


Example string: "LLM is great!"

r50k_base: 5 tokens
token integers: [3069, 44, 318, 1049, 0]
token bytes: [b'LL', b'M', b' is', b' great', b'!']

p50k_base: 5 tokens
token integers: [3069, 44, 318, 1049, 0]
token bytes: [b'LL', b'M', b' is', b' great', b'!']

cl100k_base: 5 tokens
token integers: [4178, 44, 374, 2294, 0]
token bytes: [b'LL', b'M', b' is', b' great', b'!']

Example string: "Merry Christmas"

r50k_base: 3 tokens
token integers: [44, 6996, 6786]
token bytes: [b'M', b'erry', b' Christmas']

p50k_base: 3 tokens
token integers: [44, 6996, 6786]
token bytes: [b'M', b'erry', b' Christmas']

cl100k_base: 3 tokens
token integers: [44, 5515, 10280]
token bytes: [b'M', b'erry', b' Christmas']

Example string: "春眠不觉晓"

r50k_base: 11 tokens
token integers: [23626, 98, 40367, 254, 38834, 164, 100, 231, 162, 247, 241]
token bytes: [b'\xe6\x98', b'\xa5', b'\xe7\x9c', b'\xa0', b'\xe4\xb8\x8d', b'\xe8', b'\xa7', b'\x89', b'\xe6', b'\x99', b'\x93']

p50k_base: 11 tokens
token integers: 

['�', '�', '�', '�', '不', '�', '�', '�', '�']

##  Meta llama-2 Tokenizer

Reference:
* https://github.com/facebookresearch/llama
* https://huggingface.co/meta-llama/Llama-2-7b
* https://huggingface.co/docs/hub/security-tokens

Set huggingface token into below snippet

In [ ]:
import os
import torch
from transformers import  AutoTokenizer

token = "..."
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
type(tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast

In [ ]:
tokenizer.encode("this is a dog")

[1, 445, 338, 263, 11203]

In [ ]:
[tokenizer.decode(token) for token in [1, 445, 338, 263, 11203]]


['<s>', 'this', 'is', 'a', 'dog']

In [ ]:
tokenizer.vocab["this"]

1366

In [ ]:
from transformers import LlamaTokenizerFast
tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")
tokenizer.encode("this is a dog")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

[1, 445, 338, 263, 11203]

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
from transformers import AutoTokenizer, LlamaTokenizerFast
llama_tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-alpaca-2-13b")
print(llama_tokenizer.encode("这是一只狗"))
tokenizer = LlamaTokenizerFast.from_pretrained("hfl/chinese-alpaca-2-13b")
print(tokenizer.encode("这是一只狗"))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[1, 39685, 33708, 32499]
[1, 39685, 33708, 32499]
